In [ ]:
from pathlib import Path
from tqdm import tqdm
import napari
import tifffile
import numpy as np
from cellpose import models
import skimage
from skimage import measure, exposure
from scipy.ndimage import binary_erosion, binary_dilation, label
import pyclesperanto_prototype as cle
from apoc import ObjectSegmenter, PixelClassifier
import pandas as pd
import matplotlib.pyplot as plt
from utils import count_particles_in_nuclei
import re

In [ ]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.lsm"):
    images.append(str(file_path))

Extract all the files that did pass QC and display them

In [ ]:
# Define the results you want to explore
csv_path = "./results/results_cellpdia30_sigma6_dilrad4_dnad_obj_seg_v1_gliaero6_gliathr20.csv" 
mouse_id_csv_path = "./mouse_ids.csv"

# Read the CSV file containing the analysis results
df = pd.read_csv(csv_path)

# Read the CSV file containing the sample_id data
df_mouse_id = pd.read_csv(mouse_id_csv_path, delimiter=";", encoding="UTF-8")

# Convert the index into a column
df.reset_index(inplace=True)

# Extract 'tissue_location'
df['tissue_location'] = df['filename'].str.split('_40X_').str[-1]

# Extract 'staining_id'
df['staining_id'] = df['filename'].str.extract('(\d+)_40X')

# Make sure staining_id are of the same data type before merging df and df_mouse_id
df['staining_id'] = pd.to_numeric(df['staining_id'], errors='coerce')
df_mouse_id['staining_id'] = pd.to_numeric(df_mouse_id['staining_id'], errors='coerce')

# Merge both processed_results_df and mouse_id dataframes on staining_id
merged_df = pd.merge(df, df_mouse_id, on="staining_id")

# Calculate mean area of the image occupied by glia+ signal
glia_mask_area_mean = df['%_glia+_signal'].mean() 

# Calculate mean area of the image occupied by dna_damage_+ signal
dna_damage_mask_area_mean = df['%_dna_damage_signal'].mean()

# Define a function to determine staining quality
def determine_stain_quality(value, mean_value):
    if value < (mean_value + mean_value*3):
        return "optimal"
    else:
        return "suboptimal"

# Check stain quality for glia and create another column storing optimal or suboptimal if qc_passed or not    
merged_df['glia_stain_quality_auto'] = merged_df['%_glia+_signal'].apply(lambda x: determine_stain_quality(x, glia_mask_area_mean))

# Check stain quality for dna_damage and create another column storing optimal or suboptimal if qc_passed or not 
merged_df['dna_damage_stain_quality_auto'] = merged_df['%_dna_damage_signal'].apply(lambda x: determine_stain_quality(x, dna_damage_mask_area_mean))

# Check for both stain qualities and store True qc_passed if both are optimal
merged_df['staining_qc_passed'] = (merged_df['glia_stain_quality_auto'] == 'optimal') & (merged_df['dna_damage_stain_quality_auto'] == 'optimal')

# Group the DataFrame by 'staining_id' and check if all 'staining_qc_passed' values are True, otherwise set them all to False
merged_df['staining_qc_passed'] = merged_df.groupby('staining_id')['staining_qc_passed'].transform('all')

# Now, if all 'staining_qc_passed' values for the same 'staining_id' were True, the column will remain True; otherwise, it will be False

# Remove data from images with a poor quality stain (auto QC)
auto_filtered_df = merged_df[merged_df['staining_qc_passed'] == True]

# Extract the images not passing staining_qc
qc_passed_df = merged_df[merged_df['staining_qc_passed'] == True]

print(f"{qc_passed_df.shape[0]} stains have passed QC and will be displayed")

qc_passed_list = qc_passed_df['index'].tolist()

qc_passed_df

Extract analysis settings from results.csv name and apply them to the images that passed QC

In [ ]:
# Extract analysis parameters from the CSV path
extracted_values = re.findall(r'\d+', csv_path)

# Dynamically assign the extracted values to variables
if len(extracted_values) >= 6:
    cellpose_nuclei_diameter = int(extracted_values[0])
    gaussian_sigma = int(extracted_values[1])
    dilation_radius_nuclei = int(extracted_values[2])
    dna_damage_segmenter_version = int(extracted_values[3])
    glia_nuclei_colocalization_erosion = int(extracted_values[4])
    if "_glia_sem_seg_v" in str(csv_path):
        glia_segmenter = True
    else:
        glia_segmenter = False

if glia_segmenter:
    glia_segmenter_version = int(extracted_values[5])
    glia_channel_threshold = None
    # Dinamically adjust plot titles
    title = f"cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_glia_sem_seg_v{glia_segmenter_version}"
else:
    glia_channel_threshold = int(extracted_values[5])
    glia_segmenter_version = None
    # Dinamically adjust plot titles
    title = f"cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_gliathr{glia_channel_threshold}"
    

# Print the assigned analysis parameters
print(f"Cellpose nuclei diameter: {cellpose_nuclei_diameter}")
print(f"Gaussian sigma: {gaussian_sigma}")
print(f"Dilation radius nuclei: {dilation_radius_nuclei}")
print(f"Dna damage segmenter version: {dna_damage_segmenter_version}")
print(f"Glia erosion: {glia_nuclei_colocalization_erosion}")
print(f"Glia threshold: {glia_channel_threshold}")
print(f"Glia semantic segmentation version: {glia_segmenter_version}")

Display in-notebook (using matplotlib) the images that passed QC using the above extracted analysis settings

In [ ]:
filenames_indexerror = []

model = models.Cellpose(gpu=True, model_type='nuclei')

for index in qc_passed_list:

        image = images[index]

        # Extract filename
        file_path = Path(image)
        filename = file_path.stem

        # Read the image file
        img = tifffile.imread(image)
        
        try:

            # Extract the stack containing the nuclei (0), dna_damage (1) and microglia channel (2)
            nuclei_stack = img[:, 0, :, :]
            dna_damage_stack = img[:, 1, :, :]
            microglia_stack = img[:, 2, :, :]

            # Perform maximum intensity projections
            nuclei_mip = np.max(nuclei_stack, axis=0)
            dna_damage_mip = np.max(dna_damage_stack, axis=0)
            microglia_mip = np.max(microglia_stack, axis=0)

            # Create a copy of nuclei_mip
            input_img = nuclei_mip.copy()

            # Might need to perform a Gaussian-blur before
            post_gaussian_img = skimage.filters.gaussian(input_img, sigma=gaussian_sigma)

            # Apply Contrast Stretching to improve Cellpose detection of overly bright nuclei
            p2, p98 = np.percentile(post_gaussian_img, (2, 98))
            img_rescale = exposure.rescale_intensity(post_gaussian_img, in_range=(p2, p98))

            # Predict nuclei nuclei_masks using cellpose
            nuclei_masks, flows, styles, diams = model.eval(
                img_rescale,
                diameter=cellpose_nuclei_diameter,
                channels=[0, 0],
                net_avg=False,
            )

            # Dilate nuclei to make sure the dna_damage_foci objects sit inside
            dilated_nuclei_masks = cle.dilate_labels(
                nuclei_masks, radius=dilation_radius_nuclei
            )
            # Erode dilated_nuclei_masks to obtain separate objects upon binarization
            eroded_nuclei_masks = cle.erode_labels(dilated_nuclei_masks, radius=1)

            # Set a threshold value for the pixels in microglia channel
            threshold = glia_channel_threshold  # Based on the microglia marker intensity across images

            # Create a new array with the same shape as nuclei_masks, initialized with False
            result_array = np.full_like(nuclei_masks, False, dtype=bool)

            if glia_segmenter:
                # Predict glia mask using the semantic_segmenter instead of thresholding
                segmenter = PixelClassifier(
                    opencl_filename=f"./semantic_segmenters/microglia_segmenter_v{glia_segmenter_version}.cl"
                )
                above_threshold_indices = segmenter.predict(image=microglia_mip)
                # Extract ndarray from OCLArray
                above_threshold_indices = cle.pull(above_threshold_indices)
                # Transform into a boolean array
                above_threshold_indices = above_threshold_indices > 1
            else:
                # Find indices where values in values_array are above the threshold
                above_threshold_indices = microglia_mip > threshold

            # Update the corresponding positions in the result_array based on the mask_array
            nuclei_masks_bool = nuclei_masks.astype(
                bool
            )  # Make a boolean copy of nuclei_masks to be able to use logical operators
            result_array[nuclei_masks_bool & above_threshold_indices] = True

            # Convert the boolean array to a binary array
            binary_result_array = result_array.astype(int)

            # Now, result_array contains True where both conditions are satisfied, and False otherwise
            # viewer.add_labels(binary_result_array, name=f"{filename}_nuclei+glia_coloc")

            # Erode binary_result_array to get rid of small nuclei pixels colocalizing with glia branches
            # Set the structuring element for erosion
            structuring_element = np.ones(
                (glia_nuclei_colocalization_erosion, glia_nuclei_colocalization_erosion),
                dtype=bool,
            )  # You can adjust the size and shape

            # Perform binary erosion
            eroded_array = binary_erosion(
                binary_result_array, structure=structuring_element
            )

            # Now I want to recover just the nuclei_masks that are sitting on top of binary_results_array
            labels, num_labels = measure.label(nuclei_masks, return_num=True)

            # Create an array of indices corresponding to the True values in binary_result_array
            true_indices = np.where(eroded_array)

            # Initialize an array to store labels for each processed region
            processed_region_labels = np.zeros_like(nuclei_masks, dtype=int)

            # Label index for processed regions
            label_index = 1

            # Iterate over each connected component
            for label_ in range(1, num_labels + 1):
                # Extract the region corresponding to the current label
                region = labels == label_

                # Check if any True value in binary_result_array is present in the region
                if np.any(region[true_indices]):
                    # Assign a unique label to the processed region
                    processed_region_labels[region] = label_index
                    label_index += 1

            # viewer.add_labels(processed_region_labels, name=f"{filename}_glia+_nuclei_mask")

            # Dilate processed_regions to make sure the dna_damage_foci objects sit inside
            dilated_glia_pos_nuclei = cle.dilate_labels(processed_region_labels, radius=dilation_radius_nuclei)
            # Erode processed_regions to obtain separate objects upon binarization
            eroded_glia_pos_nuclei = cle.erode_labels(dilated_glia_pos_nuclei, radius=1)

            # Apply object segmenter from APOC
            segmenter = ObjectSegmenter(
                opencl_filename=f"./object_segmenters/dna_damage_object_segmenter_v{dna_damage_segmenter_version}.cl"
            )
            dna_damage_masks = segmenter.predict(image=dna_damage_mip)

            # Erode dna_damage_masks to get rid of small artifacts
            # Set the structuring element for erosion
            structuring_element_damage = np.ones(
                (2, 2), dtype=bool
            )  # You can adjust the size and shape

            # Perform binary erosion
            eroded_dna_damage = binary_erosion(
                dna_damage_masks, structure=structuring_element_damage
            )

            # Perform binary dilation to fill holes
            dilated_dna_damage = binary_dilation(
                eroded_dna_damage, structure=structuring_element_damage
            )

            # Label the DNA damage particles
            labeled_dna_damage, num_dna_damage = label(dilated_dna_damage)

            # Label the nuclei in eroded_glia_pos_nuclei and eroded_nuclei_masks
            labeled_glia_nuclei, num_glia_nuclei = label(eroded_glia_pos_nuclei)
            labeled_nuclei, num_nuclei = label(eroded_nuclei_masks)

            # Count particles in glia nuclei
            particles_in_glia_nuclei = count_particles_in_nuclei(
                labeled_dna_damage, labeled_glia_nuclei, num_glia_nuclei
            )

            # Count particles in other nuclei
            particles_in_nuclei = count_particles_in_nuclei(
                labeled_dna_damage, labeled_nuclei, num_nuclei
            )

            # The arrays particles_in_glia_nuclei and particles_in_nuclei now contain the counts of DNA damage particles
            # in each nucleus of dilated_glia_pos_nuclei and dilated_nuclei_masks respectively
            
            # Visualize the original image and segmentation nuclei_masks
            print(filename)
            print(particles_in_glia_nuclei)
            
            plt.figure(figsize=(40, 40))
            plt.subplot(1, 7, 1)
            plt.imshow(nuclei_mip, cmap='gray')
            plt.title('Input Nuclei MIP Image')
            plt.axis("off")

            plt.subplot(1, 7, 2)
            plt.imshow(eroded_nuclei_masks, cmap='viridis')
            plt.title('Segmentation nuclei')
            plt.axis("off")

            plt.subplot(1, 7, 3)
            plt.imshow(microglia_mip, cmap='gray')
            plt.title('Input Glia MIP Image')
            plt.axis("off")

            plt.subplot(1, 7, 4)
            plt.imshow(above_threshold_indices, cmap='viridis')
            plt.title('Segmentation Glia')
            plt.axis("off")

            plt.subplot(1, 7, 5)
            plt.imshow(eroded_glia_pos_nuclei, cmap='viridis')
            plt.title('Segmentation glia+ nuclei')
            plt.axis("off")

            plt.subplot(1, 7, 6)
            plt.imshow(dna_damage_mip, cmap='gray')
            plt.title('Input DNA damage')
            plt.axis("off")

            plt.subplot(1, 7, 7)
            plt.imshow(dilated_dna_damage, cmap='gray')
            plt.title('DNA damage mask')
            plt.axis("off")

            plt.show()
            
        except IndexError:
            print(f"Filename: {filename} displays an IndexError")
            filenames_indexerror.append(filename)
            pass

In [ ]:
filenames_not_passing